In [1]:
import torch
print(torch.__version__)
print(torch.backends.mps.is_available())

2.5.1
True


In [5]:
import torch
import time

# Check device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# Simple matrix multiplication test
x = torch.rand(1000, 1000).to(device)
y = torch.rand(1000, 1000).to(device)

start = time.time()
z = torch.matmul(x, y)
print(f"Matrix multiplication took {time.time() - start:.4f} seconds")

Using device: mps
Matrix multiplication took 0.0008 seconds


In [2]:
import torch
print(torch.__version__)
print(torch.backends.mps.is_available())
import numpy
print(numpy.__version__)

2.5.1
True
2.2.3


In [4]:
import imageio.v2 as imageio

In [5]:
img_arr = imageio.imread('../dlwpt-code/data/p1ch4/image-dog/bobby.jpg')
img_arr.shape

(720, 1280, 3)

In [7]:
import torch

img = torch.from_numpy(img_arr)
out = img.permute(2, 0, 1)

In [8]:
out.shape

torch.Size([3, 720, 1280])

In [9]:
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)

In [10]:
batch.shape

torch.Size([3, 3, 256, 256])

In [12]:
import os 

data_dir = '../dlwpt-code/data/p1ch4/image-cats/'
file_names = [name for name in os.listdir(data_dir)
              if os.path.splitext(name)[-1] == '.png']

for i, file_name in enumerate(file_names):
    img_arr = imageio.imread(os.path.join(data_dir, file_name))
    img_t = torch.from_numpy(img_arr)
    img_t = img_t.permute(2, 0, 1)
    img_t = img_t[:3]
    batch[i] = img_t

torch.Size([3, 3, 256, 256])